In [ ]:
from google.colab import drive

drive._mount('/content/gdrive')

In [ ]:
!unzip "/content/gdrive/My Drive/testing.zip" -d "/content"

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet
import torch

def get_model():
    model = EfficientNet.from_pretrained('efficientnet-b3')
    model.load_state_dict(torch.load('/content/gdrive/My Drive/models/best_model_8_validation_accuracy=0.8764.pt',
                                     map_location=torch.device('cpu')))
    model.eval()
    return model

In [ ]:
import torchvision.transforms as transforms

def get_transformations():
  return transforms.Compose([transforms.Resize((224, 224)),
                             transforms.ToTensor(),
                             transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                  std=[0.229, 0.224, 0.225])
                             ])

In [ ]:
from pandas import read_csv

def get_landmarks():
    landmarks = read_csv('/content/gdrive/My Drive/landmarks with enough images.csv')['landmark']
    landmarks = landmarks.apply(str)
    return landmarks.sort_values().to_numpy()

In [ ]:
from PIL import Image

def get_image(path):
    return Image.open(path)

In [ ]:
def get_transformed_image(image, transformations):
    return transformations(image).unsqueeze(axis=0)

In [ ]:
import json
import numpy
import torch

def get_predictions(model, landmarks, image):
    with torch.no_grad():
        predictions = model(image)[0]
    predictions -= torch.min(predictions)
    best_predictions, indices = torch.topk(predictions, 3)
    best_predictions = best_predictions / torch.sum(best_predictions) * 100
    best_predictions = best_predictions.detach().numpy()
    result = []
    for index in range(3):
        result += [{
            "images": [f"https://landmark-retrieval.s3.eu-central-1.amazonaws.com/{landmarks[indices[index]]}/{image}.jpg" 
                       for image in range(10)],
            "probability": str(best_predictions[index])
        }]
    return json.dumps(result)

In [ ]:
model = get_model()
transformations = get_transformations()
landmarks = get_landmarks()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


In [ ]:
image = get_transformed_image(get_image('/content/testing/100230/09578c3160d0af20.jpg'), 
                              transformations)
get_predictions(model, landmarks, image)

'[{"images": ["https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/0.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/1.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/2.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/3.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/4.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/5.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/6.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/7.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/8.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/100230/9.jpg"], "probability": "40.75535"}, {"images": ["https://landmark-retrieval.s3.eu-central-1.amazonaws.com/13471/0.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/13471/1.jpg", "https://landmark-retrieval.s3.eu-central-1.amazonaws.com/13471/2.jpg",

In [ ]:
!pip install pytorch-ignite

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
device.type

'cuda'

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

def create_data_loader(folder, transformations, use_cuda, batch_size=32, 
                       shuffle=False):
  if folder is None or transformations is None: 
      return None
  if use_cuda:
    arguments = {
        'num_workers': 1,
        'pin_memory': True
        }
  else:
    arguments = {}
  return DataLoader(ImageFolder(folder, transform=transformations), 
                    batch_size=batch_size, shuffle=shuffle, **arguments)

In [ ]:
testing_data_loader = create_data_loader('/content/testing', transformations,
                                         use_cuda)

In [ ]:
from ignite.engine import create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from torch.nn import CrossEntropyLoss
metrics = {
    'accuracy': Accuracy(), 
    'loss': Loss(CrossEntropyLoss()),
}
testing_evaluator = create_supervised_evaluator(model.to(device), metrics=metrics,
                                                device=device)
testing_evaluator.run(testing_data_loader)
metrics = testing_evaluator.state.metrics
print(f'Testing loss: {metrics["loss"]:.3f} and '
        f'accuracy: {metrics["accuracy"]:.3f} ')

Testing loss: 0.534 and accuracy: 0.873 
